In [1]:
import numpy as np
np.random.seed(1)


In [2]:
X = np.array([[0,1,0],
              [1,0,0],
              [1,1,1],
              [0,1,1]])

y = np.array([[0,1,1,0]]).T
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [3]:
W = 2*np.random.random((3,1)) - 1
b = 0

In [4]:
z = X.dot(W) + b
A = sigmoid(z)

In [5]:
print(A)

[[0.60841366]
 [0.45860596]
 [0.3262757 ]
 [0.36375058]]


Value at risk (VaR)
Estimating the risk of a portfolio is very important to long-term capital growth. There are a lot of techniques to help manage risk, but here we'll be covering 'value at risk' (VaR).

There are two main ways to calculate VaR:

Using Monte Carlo simulation
Using the variance-covariance method
In this post, we'll focus on using method 2 (variance-covariance).

VaR provides an estimate of the maximum loss from a given portfolio over a period of time. You can calculate VaR at various confidence levels.

For example, given a VaR of 250,000 at 95% confidence, there is a 95% chance that your portfolio will lose no more than 250,000 the following day.

Note that the standard VaR calculation assumes the following:

Normal distribution of returns - VaR assumes the returns of the portfolio are normally distributed. This is of course not realistic for most assets, but allows us to develop a baseline using a much more simplistic calculation. (Modifications can be made to VaR to account for different distributions, but here we'll focus on the standard VaR calculation)
Standard market conditions - Like many financial instruments, VaR is best used for considering loss in standard markets, and is not well-suited for extreme/outlier events.
Steps to calculate the VaR of a portfolio
In order to calculate the VaR of your portfolio, you can follow the steps below:

Calculate periodic returns of the stocks in your portfolio
Create a covariance matrix based on the returns
Calculate the portfolio mean and standard deviation
(weighted based on investment levels of each stock in portolio)
Calculate the inverse of the normal cumulative distribution (PPF) with a specified confidence interval, standard deviation, and mean
Estimate the value at risk (VaR) for the portfolio by subtracting the initial investment from the calculation in step 4

In [ ]:
import pandas as pd
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
    
#Create our portfolio of equities
tickers = ['AAPL','FB', 'C', 'DIS']
 
#Set the investment weights (I arbitrarily picked for example)
weights = np.array([.25, .3, .15, .3])
 
#Set an initial investment level
initial_investment = 1000000
 
#Download closing prices
data = pdr.get_data_yahoo(tickers, start="2018-01-01", end=dt.date.today())['Close']
 
#From the closing prices, calculate periodic returns
returns = data.pct_change()

<h4>2) Create a covariance matrix based on the returns </h4>


In [ ]:
#Generate Var-Cov matrix
cov_matrix = returns.cov()
cov_matrix


<h4>3) Calculate the portfolio mean and standard deviation</h4>

In [ ]:
#Calculate mean returns for each stock
avg_rets = returns.mean()
 
#Calculate mean returns for portfolio overall, 
#using mean, using dot product formula to 
#normalize against investment weights
port_mean = avg_rets.dot(weights)
 
#Calculate portfolio standard deviation
port_stdev = np.sqrt(weights.T.dot(cov_matrix).dot(weights))
 
#Calculate mean of given investment
mean_investment = (1+port_mean) * initial_investment
             
#Calculate standard deviation of given investmnet
stdev_investment = initial_investment * port_stdev

<h4>4) Calculate the inverse of the normal cumulative distribution (PPF) with a specified confidence interval, standard deviation, and mean</h4>

In [ ]:
#Select our confidence interval (I'll choose 95% here)
conf_level1 = 0.05

#Using SciPy ppf method to generate values for the
#inverse cumulative distribution function to a normal distribution
#Plugging in the mean, standard deviation of our portfolio
#as calculated above
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html
from scipy.stats import norm
cutoff1 = norm.ppf(conf_level1, mean_investment, stdev_investment)


5) Estimate the value at risk (VaR) for the portfolio by subtracting the initial investment from the calculation in step 4

In [ ]:
#Finally, we can calculate the VaR at our confidence interval
var_1d1 = initial_investment - cutoff1
var_1d1

<p>Here we are saying with 95% confidence that the loss of our portfolio will not exceed ~$22.3k over a one day period.</p>
<h4>Value at risk over n-day time period</h4>
<p>What if we wanted to calculate this over a larger window of time? Below we can easily do that by just taking our 1 day VaR and multiplying it by the <a href="https://www.investopedia.com/articles/04/101304.asp" target="_blank">square root of the time period</a>.</p>

In [ ]:
# Calculate n Day VaR
var_array = []
num_days = int(15)
for x in range(1, num_days+1):    
    var_array.append(np.round(var_1d1 * np.sqrt(x),2))
    print(str(x) + " day VaR @ 95% confidence: " + str(np.round(var_1d1 * np.sqrt(x),2)))

# Build plot
plt.xlabel("Day #")
plt.ylabel("Max portfolio loss (USD)")
plt.title("Max portfolio loss (VaR) over 15-day period")
plt.plot(var_array, "r")


Checking distributions of our equities against normal distribution


In [ ]:
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import scipy

returns['AAPL'].hist(bins=40, normed=True,histtype="stepfilled",alpha=0.5)
x = np.linspace(port_mean - 3*port_stdev, port_mean+3*port_stdev,100)
plt.plot(x, scipy.stats.norm.pdf(x, port_mean, port_stdev), "r")
plt.title("AAPL returns (binned) vs. normal distribution")
plt.show()

In [ ]:
returns['FB'].hist(bins=40, normed=True,histtype="stepfilled",alpha=0.5)
x = np.linspace(port_mean - 3*port_stdev, port_mean+3*port_stdev,100)
plt.plot(x, scipy.stats.norm.pdf(x, port_mean, port_stdev), "r")
plt.title("FB returns (binned) vs. normal distribution")
plt.show()

In [ ]:
returns['C'].hist(bins=40, normed=True,histtype="stepfilled",alpha=0.5)
x = np.linspace(port_mean - 3*port_stdev, port_mean+3*port_stdev,100)
plt.plot(x, scipy.stats.norm.pdf(x, port_mean, port_stdev), "r")
plt.title("C returns (binned) vs. normal distribution")
plt.show()


In [ ]:
returns['DIS'].hist(bins=40, normed=True,histtype="stepfilled",alpha=0.5)
x = np.linspace(port_mean - 3*port_stdev, port_mean+3*port_stdev,100)
plt.plot(x, scipy.stats.norm.pdf(x, port_mean, port_stdev), "r")
plt.title("DIS returns (binned) vs. normal distribution")
plt.show()